<a href="https://colab.research.google.com/github/jcuesta123/03MIAR---Algoritmos-de-Optimizacion/blob/main/FranciscoJavier_CuestaArija_TrabajoPractico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos:   Francisco Javier Cuesta Arija
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>
>2. Organizar los horarios de partidos de La Liga<br>
>3. Combinar cifras y operaciones

## Descripción del problema: 1. Sesiones de doblaje

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10

Número de tomas : 30

Actores/Tomas : https://bit.ly/36D8IuK

- 1 indica que el actor participa en la toma

- 0 en caso contrario

(*) La respuesta es obligatoria





                                        

## (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




# La restricción del problema es que el número máximo de tomas en un día son 6.

Por lo tanto, sin esa restricción se podrían hacer las 30 tomas en el mismo día, y solamente habría que pagar una vez a cada uno de los diez actores. Es decir, que el coste sería el mínimo, 10 pagos por actor-día.

Haciendo las 30 tomas el mismo día, las permutaciones posibles sobre como ordenar todas las tomas teniendo en cuenta el orden serían: 30! = 2,653x10^32. No obstante, daría igual el orden ya que el coste sería el mismo.


Por otro lado, con la restricción aplicada, el mínimo de días para la grabación es necesariamente 5. Más días en cualquier caso solo incrementaría el coste.

Hay 30 tomas, por lo que cada una se va a relacionar con un día (1º-2º-3º-4º-5º). O de otra forma, hay que distribuir los 5 días entre las 30 tomas, y cada día se repite 6 veces. Es una permutación sin repeticiones.

A diferencia del cálculo anterior, ahora se van a contar las posibilidades de distribuir las 30 tomas entre los 5 días, cada uno con 6 huecos. Por lo tanto las posibles permutaciones en este caso son: 30!/(6!)^5 = 1,371x10^18

## Modelo para el espacio de soluciones<br>(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Cada toma se asigna a uno de los 5 días posibles. Se puede expresar como un vector de la dimension del número de tomas, 30, que relaciona directamente cada toma con uno de los 5 posibles días. Por tanto cada elemento de este vector puede tomar un valor entero entre 1 y 5, y tendrá exactamenete 6 elementos de cada cifra.

A continuación se muestra el codigo y su ejecución. Para leer los datos del enunciado previamente se ha descargado el excel en la misma carpeta de este programa.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_excel('tabla_actores.xlsx')   #se carga el excel con los datos
dia_tomas = [1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5]  #30 valores, 6 de cada cifra

df['dia'] = dia_tomas   #inicialmente se ponen en esta secuencia
print(df)

   Unnamed: 0  A1  A2  A3  A4  A5  A6  A7  A8  A9  A10  dia
0          T1   1   1   1   1   1   0   0   0   0    0    1
1          T2   0   0   1   1   1   0   0   0   0    0    1
2          T3   0   1   0   0   1   0   1   0   0    0    1
3          T4   1   1   0   0   0   0   1   1   0    0    1
4          T5   0   1   0   1   0   0   0   1   0    0    1
5          T6   1   1   0   1   1   0   0   0   0    0    1
6          T7   1   1   0   1   1   0   0   0   0    0    2
7          T8   1   1   0   0   0   1   0   0   0    0    2
8          T9   1   1   0   1   0   0   0   0   0    0    2
9         T10   1   1   0   0   0   1   0   0   1    0    2
10        T11   1   1   1   0   1   0   0   1   0    0    2
11        T12   1   1   1   1   0   1   0   0   0    0    2
12        T13   1   0   0   1   1   0   0   0   0    0    3
13        T14   1   0   1   0   0   1   0   0   0    0    3
14        T15   1   1   0   0   0   0   1   0   0    0    3
15        T16   0   0   0   1   0   0   

El objetivo es ordenar ese vector-columa "dia_tomas" para optimizar la función de coste del problema.

## Según el modelo para el espacio de soluciones<br>(*)¿Cual es la función objetivo?(*)¿Es un problema de maximización o minimización?

Se parte de la idea que hay que contar cuantos días acude a grabar cada actor. Solo el día que no hace ninguna toma es cuando no cobra. Por lo tanto, si los 10 actores fueran los 5 días, habría que pagar 50 sueldos actor-día.

Filtrando las filas del DataFrame por un día en concreto, salen 10 columnas de actores por 6 de las tomas de ese día. Multiplicado escalarmente cada columna de cada actor por un vector de unos de dimension 6, obtenemos cuantas tomas graba cada actor ese día.

Esto se puede recopilar en una matriz con columnas de actores por filas de días, es el resultado anteriormente descrito. Esta matriz se va a llamar "mat_dias" y se puede ver un ejemplo abajo en forma de DataFrame. Consecuentemente, las posiciones con el valor 0 indican que ese día ese actor no debe presentarse y no hay que pagarle. El número de 0 se obtiene con la funcion "np.sum(mat_dias == 0)".

Por lo tanto, la función objetivo es:  f = 50 - np.sum(mat_dias == 0). Entonces se puede decir que es un problema de minimización, aunque se podría ver también como un problema de maximizar los días en los que los actores no hacen ninguna toma.

In [ ]:
#ejemplo de la funcion mat_dias en versión dataframe para mejor visualización

df_mat_dias = pd.DataFrame(mat_dias)
df_mat_dias = df_mat_dias.astype(int)

print(df_mat_dias)
print('Hay ', np.sum(mat_dias == 0), 'ceros en la matriz')

   0  1  2  3  4  5  6  7  8  9
0  3  2  3  2  0  3  0  0  0  2
1  5  2  1  4  4  1  0  0  0  0
2  4  4  3  1  2  3  2  3  0  0
3  6  3  2  3  2  1  1  0  2  0
4  4  3  4  5  3  0  0  1  0  0
Hay  16 ceros en la matriz


Recapitulando, la idea de la función es:
1) Asignar un día a cada toma, como se ha mostrado en el apartado anterior
2) Filtrar el dataframe por el día de cada toma, recorriendo los 5 con un bucle
3) Una vez filtrado, multiplicar cada columna escalarmente por un vector de unos, cuyo resultado es el numero de tomas por actor ese dia [ columna_actor-dia @ np.ones(6) ]
4) Se recogen esos datos en la matriz "mat_dias", que para cada actor (columnas) y cada día (filas) indica cuantas tomas son necesarias
5) Se cuentan los días-actor que sean cero, y se restan a los 50 que serían los maximo posibles.

El código del algoritmo se ve en la siguiente pregunta.

## Diseña un algoritmo para resolver el problema por fuerza bruta

Para resolver el problema mediante fuerza bruta habría que comprobar todas las permutaciones posibles del vector de días de las tomas. Para esto se puede usar la función "permutations" que desarrolla todas las permutaciones posibles.

Sin embargo, al ser 30 valores hay tantas posibles combinaciones de números que ni por tiempo ni por memoria se daría abasto computacionalmente. De hecho al ejecutarlo da error, y por eso se muestran solo unos pocos ejemplos.

In [ ]:
#es inabordable, pero se muestra algunos ejemplos de como funcionaria

from itertools import permutations

#permutaciones = list(permutations(valores))   no hay memroria suficiente, peligro al ejecutar este comando
permutaciones = [(1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5),
                (4, 2, 3, 5, 2, 1, 5, 3, 2, 1, 5, 4, 3, 2, 3, 4, 1, 4, 5, 2, 1, 5, 4, 1, 5, 1, 3, 4, 3, 2),
                (5, 5, 3, 3, 5, 2, 2, 1, 5, 4, 3, 3, 4, 3, 4, 1, 5, 1, 1, 5, 3, 4, 2, 1, 1, 4, 2, 4, 2, 2)] #ejemplo

dias_minimos = 50
for permutacion in permutaciones:

    df['dia'] = permutacion
    mat_dias = np.zeros((5,10))

    for i in range(1,6):
        df_filtrado = df[df['dia'] == i]    #se filtra por días

        for j in range(1,11):
            mat_dias[i-1][j-1] = df_filtrado.iloc[:, j] @ np.ones(6)   #creacion matriz mat_dias

    if (50 - np.sum(mat_dias == 0)) < dias_minimos:    #solo se guarda el resultado si es mejor que el actual
        dias_minimos = 50 - np.sum(mat_dias == 0)
        print(dias_minimos, permutacion)

38 (1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5)
36 (4, 2, 3, 5, 2, 1, 5, 3, 2, 1, 5, 4, 3, 2, 3, 4, 1, 4, 5, 2, 1, 5, 4, 1, 5, 1, 3, 4, 3, 2)
34 (5, 5, 3, 3, 5, 2, 2, 1, 5, 4, 3, 3, 4, 3, 4, 1, 5, 1, 1, 5, 3, 4, 2, 1, 1, 4, 2, 4, 2, 2)


## Calcula la complejidad del algoritmo por fuerza bruta

Por el número de posibilidades calculado se observa que es un porblema de orden factorial O(n!). Escala tan rápido que solo es viable para valores pequeños de la dimension n.

## (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

El principal escollo que afronta un algoritmo por fuerza bruta es la desboradante cantidad de operaciones que debe realizar en casos grandes como este. Si se puede reducir el número de combinaciones totales, entonces se facilitara la tarea en tanto que haya menos posibilidades.

Usando las técnicas de ramificación y poda podemos añadir restricciones para descartar combinaciones que no van a ser óptimas. Por ejemplo, idealmente un actor que tenga 6 o menos tomas podría acudir solo un día para reducir el coste.

En este caso, el actor 10 solo aparece en las tomas 16 y 25. Si se impone que las estas tomas se hagan el mismo día, habrá 5 combinaciones en lugar de 25. Si se aplica a más actores, la simplificación será mayor, pero tampoco se puede asegurar que no se pueda llegar a descartar así la mejor solución posible, que a priori es completamente desconocida.

A continuación se presenta el codigo que implementaría esta restricción. Esatría incompleto, pero no se ejecuta por la razón de antes, no hay memoria suficiente para trabajar por fuerza bruta.

In [ ]:
#se generan las restricciones para que el actor 10 solo acuda un día

mat_a10 = []
for i in range(5):
    dia_tomas = [1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5]

    del dia_tomas[i*6:i*6+2]
    dia_tomas.insert(15, i+1) #T16
    dia_tomas.insert(24, i+1) #T25

    mat_a10.append(dia_tomas)

#OJO aqui se deberían permutar para volver a aplicar el algoritmo de la fuerza bruta

dias_minimos = 50
for permutacion in mat_a10:

    df['dia'] = permutacion
    mat_dias = np.zeros((5,10))

    for i in range(1,6):
        df_filtrado = df[df['dia'] == i]

        for j in range(1,11):
            mat_dias[i-1][j-1] = df_filtrado.iloc[:, j] @ np.ones(6)

    if (50 - np.sum(mat_dias == 0)) < dias_minimos:
        dias_minimos = 50 - np.sum(mat_dias == 0)
        print(dias_minimos, permutacion)

36 [1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 1, 3, 4, 4, 4, 4, 4, 4, 5, 1, 5, 5, 5, 5, 5]
35 [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 3, 4, 4, 4, 4, 4, 4, 5, 2, 5, 5, 5, 5, 5]


Se observa como se ha generado un vector de los días de las tomas que consigue que que el actor 10 solo vaya un día. No obstante, el mismo concepto se podría aplicar para más actores.

## (*)Calcula la complejidad del algoritmo

Dependiendo de cuantas restricciones se impusieran, el orden bajaría más o menos. Al imponer que un elemento del vector sea necesariamente igual que el otro, dia_tomas[15] == dia_tomas[24], se reduce un grado y la complejidad seria O(29!) en lugar de O(30!).

En principio, cada restricción de este tipo rebaría un grado el orden de complejidad, siempre y cuando estas fueran compatibles con el enunciado origina. La complejidad seria tal que O((n-k)!), siendo k el número de restricciones del tipo descrito. Y aun pudiendo ser compatibles, cabría esperar que el añadir restricciones artificiales podría impedir que se lleguara a la solución óptima, por lo que habría que ser cuidadosos.

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Ahora el vector se combina de forma aleatoria a partir de la semilla incial. Se puede emplear la función "random.shuffle" para permutar de manera sencilla los elementos del vector.

In [ ]:
import random

dia_tomas = [1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5]
random.shuffle(dia_tomas)

print(dia_tomas)

[3, 4, 4, 5, 3, 3, 2, 3, 4, 1, 2, 5, 4, 1, 1, 2, 2, 2, 5, 5, 4, 3, 1, 1, 3, 2, 5, 1, 5, 4]


## Aplica el algoritmo al juego de datos generado

A continuación se introduce un vector aleatorio en el algoritmo de antes. Lo mejor de esto es que no se necesita crear una lista de todas las permutaciones del vector, por lo que no se generan problemas de memoria. Además permite explorar areas diversas más rápidamente.

Para obtener buenos resultados lo mejor sería hacer el mayor número de intentos. No obstante, esto incrementa el tiempo de cálculo. Para una estimación aceptable se ha elegido analizar 10 mil casos, que tarda unos cincuenta segundos.

In [ ]:
import time

dia_tomas = [1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5]

dias_minimos = 51

tiempo_inicio = time.time()

for k in range(10000): #OJO mas de 1000 empieza a tardar

    random.shuffle(dia_tomas)
    df['dia'] = dia_tomas

    mat_dias = np.zeros((5,10))

    for i in range(1,6):
        df_filtrado = df[df['dia'] == i]

        for j in range(1,11):
            mat_dias[i-1][j-1] = df_filtrado.iloc[:, j] @ np.ones(6)

    if (50 - np.sum(mat_dias == 0)) < dias_minimos:
        dias_minimos = 50 - np.sum(mat_dias == 0)
        print(dias_minimos, dia_tomas)

tiempo_fin = time.time()

# Calcula el tiempo transcurrido
tiempo_transcurrido = tiempo_fin - tiempo_inicio

print(f"Tiempo transcurrido: {tiempo_transcurrido} segundos")

38 [5, 2, 1, 4, 1, 3, 2, 2, 4, 3, 5, 2, 1, 1, 1, 4, 2, 3, 5, 4, 4, 4, 5, 5, 3, 3, 2, 1, 3, 5]
37 [4, 5, 4, 1, 2, 5, 4, 3, 1, 3, 1, 3, 3, 5, 4, 2, 3, 1, 5, 5, 3, 2, 2, 2, 5, 4, 4, 1, 1, 2]
36 [2, 3, 5, 5, 1, 3, 1, 1, 2, 4, 1, 4, 1, 1, 2, 3, 4, 4, 4, 2, 5, 2, 5, 3, 5, 5, 3, 2, 3, 4]
35 [2, 2, 5, 4, 4, 5, 3, 1, 4, 3, 5, 3, 5, 4, 4, 3, 1, 2, 5, 3, 1, 1, 1, 2, 5, 3, 2, 4, 2, 1]
34 [2, 2, 3, 3, 4, 5, 5, 5, 1, 3, 4, 2, 5, 2, 4, 1, 3, 2, 3, 4, 5, 1, 5, 4, 1, 3, 4, 1, 2, 1]
33 [2, 3, 5, 1, 1, 3, 2, 4, 1, 4, 5, 4, 4, 5, 5, 2, 3, 3, 1, 1, 5, 4, 2, 5, 2, 4, 1, 3, 3, 2]
Tiempo transcurrido: 52.57889699935913 segundos


En los primeros segundos encuentra soluciones cada vez mejores. Pero como se comprueba, que cada vez es más lento encontrando soluciones mejores, y en este caso al llegar a 33, ya no mejora, aunque haya estado mas de la mitad del tiempo en este punto.

Por otra parte, se quiere ver si es cierto que agrupando las tomas de los actores que tienen pocas se mejoran los resultados. Para el vector del mejor resultado se comprueba que la hipotesis parece acertada, ya que tanto el actor 10 (tomas 16 y 25) como el actor 9 (tomas 10 y 26) se realizan sus dos tomas el mismo día (días 2 y 4 respectivamente).

In [ ]:
aux = [2, 3, 5, 1, 1, 3, 2, 4, 1, 4, 5, 4, 4, 5, 5, 2, 3, 3, 1, 1, 5, 4, 2, 5, 2, 4, 1, 3, 3, 2]
print(aux[15],aux[24])
print(aux[9],aux[25])

2 2
4 4


## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Reyero, R., ALGORITMOS DE OPTIMIZACIÓN, apuntes del MÁSTER UNIVERSITARIO EN INTELIGENCIA ARTIFICIAL

## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Existen varias formas en las que es esperable mejorar la solución obtenida del problema:
- Ampliar la búsqueda de valores aleatorios, a costa de incrementar el tiempo de computación: El inconveniente es que encontrar una solución mejor cada vez es mucho más costoso.
- Implementar métodos heurísticos: El método aleatorio suele ser muy poco eficiente.
- Usar un código más eficiente: por ejemplo, el uso de dataframes es poco eficiente, sin embargo la biblioteca de numpy debería ofrecer recursos que optimizan mucho mejor los cálculos necesarios.
- Imponer restricciones que acoten considerablemente el rango de posibilidades del problema: Se detalla en la siguiente caja.

Ahora se va a volver a aplicar el algoritmo, pero imponiendo varias restricciones. Cada uno de los actores 7, 8, 9 y 10 solamente necesitan ir un día. Si además fijamos esos días, siendo el primero y el segundo, acotamos significativamente las posibles soluciones. Ya no hay que permutar 30 elementos sino 20.

El día 1 se asigna para las tomas de los actores 9 y 10, y el 2 para las de 7 y 8. El resultado, igualmente para 10 mil intentos es el siguiente.

In [ ]:
import time

dias_minimos = 51

tiempo_inicio = time.time()

for k in range(10000): #OJO mas de 1000 empieza a tardar

    dia_tomas_mod = [1,1,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5,5,5]
    random.shuffle(dia_tomas_mod)

    dia_tomas_mod.insert(2, 2) #T3
    dia_tomas_mod.insert(3, 2) #T4
    dia_tomas_mod.insert(4, 2) #T5
    dia_tomas_mod.insert(9, 1) #T10
    dia_tomas_mod.insert(10, 2) #T11
    dia_tomas_mod.insert(14, 2) #T15
    dia_tomas_mod.insert(15, 1) #T16
    dia_tomas_mod.insert(20, 2) #T21
    dia_tomas_mod.insert(24, 1) #T25
    dia_tomas_mod.insert(25, 1) #T26

    df['dia'] = dia_tomas_mod

    mat_dias = np.zeros((5,10))

    for i in range(1,6):
        df_filtrado = df[df['dia'] == i]

        for j in range(1,11):
            mat_dias[i-1][j-1] = df_filtrado.iloc[:, j] @ np.ones(6)

    if (50 - np.sum(mat_dias == 0)) < dias_minimos:
        dias_minimos = 50 - np.sum(mat_dias == 0)
        print(dias_minimos, dia_tomas_mod)

tiempo_fin = time.time()

# Calcula el tiempo transcurrido
tiempo_transcurrido = tiempo_fin - tiempo_inicio

print(f"Tiempo transcurrido: {tiempo_transcurrido} segundos")

34 [3, 5, 2, 2, 2, 4, 4, 4, 1, 1, 2, 5, 5, 3, 2, 1, 4, 5, 3, 1, 2, 3, 5, 3, 1, 1, 5, 4, 3, 4]
32 [4, 4, 2, 2, 2, 3, 4, 4, 1, 1, 2, 1, 4, 5, 2, 1, 3, 5, 5, 5, 2, 3, 3, 4, 1, 1, 3, 5, 5, 3]
31 [4, 4, 2, 2, 2, 5, 5, 1, 4, 1, 2, 5, 4, 3, 2, 1, 3, 3, 3, 4, 2, 5, 3, 3, 1, 1, 1, 5, 4, 5]
30 [3, 5, 2, 2, 2, 3, 1, 4, 3, 1, 2, 3, 5, 4, 2, 1, 4, 4, 3, 5, 2, 3, 4, 4, 1, 1, 5, 5, 1, 5]
Tiempo transcurrido: 52.48275542259216 segundos


En el mismo tiempo, se ha conseguido una solución notablemente mejor (30 vs 33). Esto se debe a haber reducido las posibilidades en varios órdenes de magnitud. La mejora es evidente, pero también es lógico plantearse si esas restricciones podrían haber bloqueado el alcance de la desconocida solución óptima.